# Práctica 6: Introducción a sensibilidad

## Librerias

In [1]:
import numpy as np
import pandas as pd 

## Simplex

Función donde se implementa el algoritmo Simplex para optimización de problemas de programación lineal en la forma estandar.

* El código fue adapatado de: https://github.com/mmolteratx/Simplex



In [2]:
# ------------------------------------------------------------------
#                      Funciones Auxiliares
# ------------------------------------------------------------------


def getTableau(c, A, b):
  # Construct starting tableau
  c[0:len(c)] = -1 * c[0:len(c)]

  t2 = np.array([None, 0])
  numVar = len(c)
  numSlack = len(A)
    
  t2 = np.hstack(([None], c, [0]))
  
  basis = np.array([0] * numSlack)
  
  for i in range(0, len(basis)):
      basis[i] = numVar + i
      
  t1 = np.hstack((np.transpose([basis]), A, np.transpose([b])))
  
  tableau = np.vstack((t1, t2))
  
  tableau = np.array(tableau, dtype ='float')
  
  return tableau

# ------------------------------------------------------------------
#                      Funcion Simplex
# ------------------------------------------------------------------

def simplex(f, A, b, print_iter=True):
  for i in range(len(f)):
    f[i] = -1 * f[i]
    
  # Build Tableu
  tableau = getTableau(f, A, b)
        
  # Assume initial basis is not optimal
  optimal = False

  # Keep track of iterations for display
  iter = 1

  while not optimal:
              
    # Look for direction of decreased cost
    for cost in tableau[-1, 1:-1]:
      if cost < 0:
        optimal = False
        break
      optimal = True

    # If all directions result in decreased cost SBF is optimal
    if optimal: 
      break
      
    # nth variable enters basis, account for tableau indexing
    n = tableau[-1, 1:-1].tolist().index(np.amin(tableau[-1, 1:-1])) + 1 # PUEDE QUE NO SEA 2

    # Minimum ratio test, rth variable leaves basis 
    minimum = 99999
    r = -1
    
    for i in range(0, len(tableau)-1): 
      if (tableau[i, n] > 0):
        val = tableau[i, -1]/tableau[i, n]
        if val<minimum: 
          minimum = val 
          r = i
                    
    pivot = tableau[r, n] 
    
    # Perform row operations 
    # Divide the pivot row with the pivot element 
    tableau[r, 1:] = tableau[r, 1:] / pivot 
    
    # Pivot other rows
    for i in range(0, len(tableau)): 
      if i != r:
        mult = tableau[i, n] / tableau[r, n]
        tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 

    # New basic variable 
    tableau[r, 0] = n - 1
    iter += 1
      
  x = np.array([0] * len(f), dtype = float)
  # Save coefficients
  for key in range(0, (len(tableau))):
      if (tableau[key, 0] < len(f)):
          x[int(tableau[key, 0])] = tableau[key, -1]
  
  optimalValue = -1 * tableau[-1,-1]

  return(x,optimalValue)

##  Problema Cremheladito

Problema:
$$ \max f(x) = 1.1 x_1 + 1.0 x_2 + 0.9 x_3 + 0.92 x_4 + 1.2 x_5 $$

$$ \begin{array}{rl}
  0.45x_1 + 0.5x_2 + 0.4x_3 + 0.4x_4 + 0.42x_5 & \leq 240 \\
  0.5x_1 + 0.4x_2 + 0.4x_3 + 0.4x_4 + 0.2x_5 & \leq 190 \\
  0.1x_1 + 0.15x_2 + 0.3x_3 + 0.2x_4 + 0.1x_5 & \leq 80 \\
  x_1,x_2,x_3,x_4,x_5 & \geq 0 \\
 \end{array}$$

 Forma Estándar:
 $$ \min f(x) = -1.1 x_1 - 1.0 x_2 - 0.9 x_3 - 0.92 x_4 - 1.2 x_5  $$

$$ \begin{array}{rl}
  0.45x_1 + 0.5x_2 + 0.4x_3 + 0.4x_4 + 0.42x_5 + s_1 & = 240 \\
  0.5x_1 + 0.4x_2 + 0.4x_3 + 0.4x_4 + 0.2x_5 + s_2 & = 190 \\
  0.1x_1 + 0.15x_2 + 0.3x_3 + 0.2x_4 + 0.1x_5 + s_3 & =80 \\
  x_1,x_2,x_3,x_4,x_5 & \geq 0 \\
 \end{array}$$

**Punto A**

Análisis sensibilidad variando una variable básica

In [8]:
# Probar función
f2 = np.array([-1.1, -1.0, -0.9, -0.92, -1.2, 0, 0, 0])
A2 = np.array([[0.45, 0.5, 0.4, 0.4, 0.42, 1, 0, 0], [0.5, 0.4, 0.4, 0.4, 0.2, 0, 1, 0], [0.1, 0.15, 0.3, 0.2, 0.1, 0, 0, 1]])
b2 = np.array([240, 190, 80])

#Arreglos para el dataframe
Sol = []
Vco = []

#Cuantificación solución y valor óptimo variando los coeficientes variables basicas
sens = np.arange(0, 2.1, 0.1)
for i in range(0,5):
    for j in range(len(sens)):
        f2[i] = -1*sens[j]
        Sol.append(simplex(f2, A2, b2)[0])
        Vco.append(-1*simplex(f2, A2, b2)[1])
        f2 = np.array([-1.1, -1.0, -0.9, -0.92, -1.2, 0, 0, 0])
        
#Para X5
i = 4
print('Análisis Sensibilidad: X' + str(i+1))
data = {"Coeficiente Variable Básica": sens,
            "Solución a las Variables Básicas": Sol[i*len(sens):(i+1)*len(sens)],
           "Valor Función Costo": Vco[i*len(sens):(i+1)*len(sens)]} 

#Visualización en un dataframe
df = pd.DataFrame(data)
df.head(21)

Análisis Sensibilidad: X5


,Coeficiente Variable Básica,Solución a las Variables Básicas,Valor Función Costo
0,0.0,"[0.0, 475.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",475.000000
1,0.1,"[0.0, 475.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",475.000000
2,0.2,"[0.0, 475.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",475.000000
3,0.3,"[0.0, 475.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",475.000000
4,0.4,"[0.0, 475.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",475.000000
5,0.5,"[0.0, 475.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",475.000000
6,0.6,"[0.0, 467.64705882352933, 0.0, 0.0, 14.7058823...",476.470588
7,0.7,"[265.0, 0.0, 0.0, 0.0, 287.50000000000006, 0.0...",492.750000
8,0.8,"[265.0, 0.0, 0.0, 0.0, 287.50000000000006, 0.0...",521.500000
9,0.9,"[265.0, 0.0, 0.0, 0.0, 287.50000000000006, 0.0...",550.250000


De acuerdo a la anterior tabla, notamos que al variar el coeficiente de utilidad de $X_{5}$ relacionado a la función costo no hay un cambió en el valor esperado para la solución y el óptimo para aquellos valores que sean mayores o iguales a 1,1. Esto se debe a que $X_{5}$ tiene el mayor coeficiente entre las variables de decisión en la función costo con 1,2, mientras que el segundo mayor coeficiente es 1,1 que corresponde a $X_{1}$. Es por esto que para valores inferiores a 1,1 en la variable básica existe una alteración en el sistema ya que $X_{5}$ deja de ser la única variable que incide notablemente en la función costo, por el contrario, pierde incidencia implicando que otras variables tomen mayor relevancia en la solución del sistema y el valor óptimo. 

<br>

Por ejemplo, $X_{1}$ tiene un mayor protagonismo para valores cercanos entre [0.6-1,1], en donde esta variable pasa a ser una solución básica. En el caso que se obtengan valores inferiores a 0,6 , la variable $X_{2}$ pasa a ser la única solución básica.


**Punto B**

Análisis sensibilidad variando una variable no básica


In [9]:
#Para X1
i = 0
print('Análisis Sensibilidad: X' + str(i+1))
data = {"Coeficiente Variable Básica": sens,
            "Solución a las Variables Básicas": Sol[i*len(sens):(i+1)*len(sens)],
           "Valor Función Costo": Vco[i*len(sens):(i+1)*len(sens)]} 

#Visualización en un dataframe
df = pd.DataFrame(data)
df.head(21)

Análisis Sensibilidad: X1


,Coeficiente Variable Básica,Solución a las Variables Básicas,Valor Función Costo
0,0.0,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
1,0.1,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
2,0.2,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
3,0.3,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
4,0.4,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
5,0.5,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
6,0.6,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
7,0.7,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
8,0.8,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
9,0.9,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286


Tal y como se puede ver en la tabla, no existe un cambio notable o diferente al esperado siempre y cuando el coeficiente que acompaña a $X_{1}$, que es su utilidad y afecta directamente la solución óptima, no supere el valor la utilidad de la variable básica en condiciones normales, es decir, el lulo o $X_{5}$. Como en el problema original $X_{5}$ tiene un coeficiente de utilidad de 1,2 implica que para valores inferiores o iguales a 1,2 para la variable $X_{1}$ no alterará la solución del sistema y en consecuencia su valor óptimo. No obstante, cuando el coeficiente de $X_{1}$ supera la magnitud de 1,2 implica que genera que $X_{1}$ tome una mayor relevancia en el sistema y que no sea únicamente $X_{5}$ la que tenga impacto en la función costo. 

<br>

Al solucionar el sistema $X_{1}$ deja de ser diferente de 0 y se vuelve una solución básica factible, impactando directamente en el valor de la solución óptima.


**Punto C**

Análisis de sensibilidad variando una de las restricciones

In [10]:
# Probar función
f2 = np.array([-1.1, -1.0, -0.9, -0.92, -1.2, 0, 0, 0])
A2 = np.array([[0.45, 0.5, 0.4, 0.4, 0.42, 1, 0, 0], [0.5, 0.4, 0.4, 0.4, 0.2, 0, 1, 0], [0.1, 0.15, 0.3, 0.2, 0.1, 0, 0, 1]])
b2 = np.array([240, 190, 80])

# Arreglo para cear el dataframe
Sol = []
Vco = []

# Cálculo soluciones y valores optimos variando los coecientes de fcosto
sens = np.arange(0, 124, 1)

# Iteración valor de la restricción de la ecuación 2
for i in range(len(sens)):
    b2[1] = sens[i]
    Sol.append(simplex(f2, A2, b2)[0])
    Vco.append(-1*simplex(f2, A2, b2)[1])
        
# Desarollo 
i = 0
data = {"Coeficiente Variable Básica: Ecuación 2": sens,
            "Solución a las Variables Básicas": Sol[i*len(sens):(i+1)*len(sens)],
           "Valor Función Costo": Vco[i*len(sens):(i+1)*len(sens)]} 

# Visualización en un dataframe
df = pd.DataFrame(data)
df.head(400)

,Coeficiente Variable Básica: Ecuación 2,Solución a las Variables Básicas,Valor Función Costo
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000
1,1,"[0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0]",6.000000
2,2,"[0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0]",12.000000
3,3,"[0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0]",18.000000
4,4,"[0.0, 0.0, 0.0, 0.0, 20.0, 0.0, 0.0, 0.0]",24.000000
...,...,...,...
119,119,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
120,120,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
121,121,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286
122,122,"[0.0, 0.0, 0.0, 0.0, 571.4285714285714, 0.0, 0...",685.714286


Por último, se modificó el valor de la restricción de la ecuación 2. Originalmente este valor era de 190 y se decidió variarlo entre [180-200] con el propósito de analizar cómo alteraba la solución del sistema y el valor optimo bajando y aumentando dicha magnitud. No obstante, no se presentó ninguna alteración en ninguna de estos dos aspectos. Esto indicaba que con el valor de la solución básica factible se encontraba en un valor muy por debajo de 190, más específicamente, con dichos valores se obtenía una magnitud de 114,4. Nótese que se decidió iterar, disminuyendo y aumentando con este corte, por lo que se halló en la tabla anterior que no hay alteración del sistema si la restricción se establece en un valor mayor a 114. Mientras que de tener una magnitud menor, ya hay una afectación directa en la solución del sistema de $X_{5}$. 

<br>

Como resultado, existe una equivalencia en el comportamiento de la magnitud del valor de esta variable de decisión, en donde alcanza su valor máximo en valores por encima de 114, mientras que empieza a disminuir conforme tiende a 0 debido a que la restricción condiciona aun más el sistema. Asimismo, se mantiene el comportamiento de que solamente $X_{5}$ es diferente de 0, por lo que sigue siendo la única variable básica factible.
